In [0]:

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/oauth2/token"}

dbutils.fs.mount(
source = "abfss://olympicdata@olympicdatahemanth.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/olympics",
extra_configs = configs)

True

In [0]:
%fs
ls "/mnt/olympics"

path,name,size,modificationTime
dbfs:/mnt/olympics/raw-data/,raw-data/,0,1695062419000
dbfs:/mnt/olympics/transformed-data/,transformed-data/,0,1695062430000


In [0]:
spark

In [0]:
athletes = spark.read.format("csv").option("header","true").load("/mnt/olympics/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").load("/mnt/olympics/raw-data/coaches.csv")
EntriesGender = spark.read.format("csv").option("header","true").load("/mnt/olympics/raw-data/EntriesGender.csv")
medals = spark.read.format("csv").option("header","true").load("/mnt/olympics/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").load("/mnt/olympics/raw-data/teams.csv")

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
EntriesGender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
EntriesGender.printSchema()


root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
EntriesGender= EntriesGender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
        .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show()

+------------------+----+
|      Team_Country|Gold|
+------------------+----+
|            Mexico|   0|
|          Colombia|   0|
|        Azerbaijan|   0|
|Dominican Republic|   0|
|           Armenia|   0|
|        Kyrgyzstan|   0|
|          Mongolia|   0|
|         Argentina|   0|
|        San Marino|   0|
|            Jordan|   0|
|          Malaysia|   0|
|           Nigeria|   0|
|           Bahrain|   0|
|      Saudi Arabia|   0|
|         Lithuania|   0|
|   North Macedonia|   0|
|           Namibia|   0|
|      Turkmenistan|   0|
|        Kazakhstan|   0|
|           Finland|   0|
+------------------+----+
only showing top 20 rows

